In [ ]:
# ---------------------------
# 1. Prerequisites:
# ---------------------------
%pip install -r requirements.txt 


In [ ]:
# ---------------------------
# 2. Importation des bibliothèques
# ---------------------------

import pandas as pd                         # Manipulation de données tabulaires (CSV, DataFrame…)
import torch                                # Bibliothèque PyTorch pour deep learning
import torch.nn as nn                       # Couches de réseaux neuronaux
import torch.nn.functional as F             # Fonctions utiles (activations, pertes, normalisation…)
import random                               # Générateur aléatoire
import math                                 # Fonctions mathématiques (log, ceil…)
import dotenv                               # Gestion des variables d'environnement
import os                                   # Gestion des fichiers et dossiers
from dotenv import load_dotenv,find_dotenv  # Charger variables d'environnement depuis .env
from supabase import create_client, Client  # Client Supabase pour Python
import boto3                                # Client AWS S3 pour Python
import io                                   # Gestion des flux d'entrée/sortie                         
import json                                 # Manipulation de données JSON
import gcsfs                                # Système de fichiers pour Google Cloud Storage
print("Bibliothèques importées")

# Localiser et recharger le .env
dotenv_path = find_dotenv()  # trouve le .env dans ton projet
load_dotenv(dotenv_path, override=True)  # override=True force le remplacement

# Vérification des variables d'environnement
SUPABASE_BUCKET = os.getenv("SUPABASE_BUCKET") 
print("Bucket utilisé :", SUPABASE_BUCKET) 

In [ ]:
# ---------------------------
# 3. Connexion à Supabase
# ---------------------------

#Configuration Supabase
SUPABASE_URL = os.getenv("SUPABASE_URL") 
SUPABASE_KEY = os.getenv("SUPABASE_KEY") 
SUPABASE_BUCKET = os.getenv("SUPABASE_BUCKET") 
file_name = os.getenv("file_name") 

# Connexion à Supabase
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY) 
print("Connecté à Supabase") 

In [ ]:
# ---------------------------
# 4. Charger et lire le CSV "df_competence_rome_eda_v2" depuis Supabase Storage
# ---------------------------

def read_csv_from_supabase(file_name, bucket_name=SUPABASE_BUCKET): 
    try:
        # Télécharger le fichier depuis le bucket Supabase
        response = supabase.storage.from_(bucket_name).download(file_name) 
        # -> "response" contient les données brutes du fichier (bytes)

        # Charger le CSV dans un DataFrame Pandas
        df = pd.read_csv(io.BytesIO(response)) 
        # On convertit d'abord les bytes en un flux mémoire (io.BytesIO), 
        # puis Pandas lit le CSV et retourne un DataFrame

        return df

    except Exception as ex:
        print("Erreur de lecture du fichier depuis Supabase Storage :")
        print("->", ex)
        return None


# Test de lecture 
df = read_csv_from_supabase("df_competence_rome_eda_v2.csv")

if df is not None:
    print("Lecture du fichier réussi :")
    print(f"-> CSV: {df.shape[0]} lignes, {df.shape[1]} colonnes")
    # df.shape renvoie (nb_lignes, nb_colonnes)

    # Préparation des dictionnaires utiles

    # Vocabulaire des compétences (chaque compétence unique reçoit un ID numérique)
    skills_vocab = {s: i for i, s in enumerate(df['code_ogr_competence'].unique())}

    # Vocabulaire des métiers (chaque code ROME reçoit un ID numérique)
    jobs_vocab   = {j: i for i, j in enumerate(df['code_rome'].unique())}

    # Dictionnaire : code ROME → libellé du métier
    job_labels   = df.drop_duplicates('code_rome').set_index('code_rome')['libelle_rome'].to_dict()

    # Dictionnaire : code ROME → liste des compétences associées
    job_to_skills = df.groupby('code_rome')['code_ogr_competence'].apply(list).to_dict()

    # Dictionnaire : code compétence → libellé compétence
    skill_to_label = df.drop_duplicates('code_ogr_competence').set_index('code_ogr_competence')['libelle_competence'].to_dict()

    # Statistiques
    n_skills = len(skills_vocab)  # nombre total de compétences uniques
    n_jobs = len(jobs_vocab)      # nombre total de métiers uniques

    # Pondération simple (inspirée TF-IDF)
    # Compter la fréquence de chaque compétence dans le DataFrame
    skill_freq = df['code_ogr_competence'].value_counts().to_dict()

    # Calculer un poids pour chaque compétence : 
    # plus une compétence est fréquente, plus son poids est faible
    skill_weight = {s: 1.0 / math.log(1 + f) for s, f in skill_freq.items()}

    print(f"{n_skills} compétences et {n_jobs} métiers chargés depuis CSV")

else:
    print("Impossible de charger le fichier.")



In [ ]:
# ---------------------------
# 2. Modèle Transformer
# ---------------------------
class JobProfileTransformer(nn.Module):
    def __init__(self, n_skills, n_jobs, emb_dim=64, n_heads=4, n_layers=2, max_len=88):
        super().__init__()

        # Embedding des compétences (chaque compétence reçoit un vecteur de dimension emb_dim)
        self.skill_emb = nn.Embedding(n_skills, emb_dim)

        # Encodage positionnel appris (pour donner une notion d’ordre dans la séquence)
        self.pos_emb = nn.Parameter(torch.randn(1, max_len, emb_dim))

        # Définition d’une couche de Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_dim,       # dimension des embeddings
            nhead=n_heads,         # nombre de têtes d’attention
            dim_feedforward=256,   # taille du réseau feedforward interne
            batch_first=True       # batch en première dimension (batch, seq_len, emb_dim)
        )

        # Empilement de plusieurs couches d’encodeur
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        # Embedding des métiers (chaque métier reçoit aussi un vecteur)
        self.job_emb = nn.Embedding(n_jobs, emb_dim)


    # --------------------------------------------------------
    # Encode un "profil de compétences" en vecteur latent
    # --------------------------------------------------------
    def encode_profile(self, skills, weights=None):
        batch_size, seq_len = skills.shape

        # Gestion des séquences plus longues que max_len (on répète pos_emb si nécessaire)
        if seq_len > self.pos_emb.size(1):
            pos_emb = self.pos_emb.repeat(1, math.ceil(seq_len / self.pos_emb.size(1)), 1)[:, :seq_len, :]
        else:
            pos_emb = self.pos_emb[:, :seq_len, :]

        # Embedding des compétences + ajout de l’encodage positionnel
        skills_emb = self.skill_emb(skills) + pos_emb  

        # Pondération optionnelle (par ex. TF-IDF pour valoriser les compétences rares)
        if weights is not None:
            skills_emb = skills_emb * weights.unsqueeze(-1)  

        # Masque pour ignorer le padding (compétence codée par 0)
        mask = (skills == 0)  

        # Passage dans le Transformer Encoder
        v = self.encoder(skills_emb, src_key_padding_mask=mask)

        # Moyenne des vecteurs de la séquence pour obtenir une seule représentation
        v = v.mean(dim=1)  

        # Normalisation L2 → vecteur de norme 1 (utile pour la similarité cosinus)
        return F.normalize(v, dim=1)


    # --------------------------------------------------------
    # Encode un métier en vecteur latent
    # --------------------------------------------------------
    def encode_job(self, job_ids):
        v = self.job_emb(job_ids)      # Embedding métier
        return F.normalize(v, dim=1)   # Normalisation L2


print("Modèle Transformer défini")


In [ ]:
# ---------------------------
# 3. Génération batch
# ---------------------------
def make_batch(batch_size=128, n_neg=20, max_mask_ratio=0.3, n_profiles_per_job=5):
    # Listes pour stocker les données
    skills_batch, weights_batch, pos_jobs_batch, neg_jobs_batch = [], [], [], []

    # Liste de tous les métiers disponibles (clés du vocabulaire)
    jobs_list = list(jobs_vocab.keys())

    # Génération de batch_size exemples
    for _ in range(batch_size):
        job = random.choice(jobs_list)                 # Tirage aléatoire d’un métier
        comps = job_to_skills[job]                     # Liste des compétences associées à ce métier

        # Générer plusieurs "profils" pour ce métier
        for _ in range(n_profiles_per_job):
            # Masquage partiel des compétences
            n_mask = random.randint(0, int(len(comps) * max_mask_ratio)) # Nb de compétences à masquer
            masked = random.sample(comps, k=max(1, len(comps) - n_mask)) # Séquence de compétences gardées

            # Ajout des compétences encodées en indices
            skills_batch.append([skills_vocab[c] for c in masked])

            # Ajout des poids associés (ex. pondération TF-IDF)
            weights_batch.append([skill_weight[c] for c in masked])

            # Label positif = le métier d’origine
            pos_jobs_batch.append(jobs_vocab[job])

            # Tirage de métiers négatifs (métiers différents du métier choisi)
            negs = random.sample([j for j in jobs_list if j != job], n_neg)
            neg_jobs_batch.append([jobs_vocab[j] for j in negs])

    # Déterminer la longueur maximale de séquence de compétences
    max_len = max(len(s) for s in skills_batch)

    # Padding pour que toutes les séquences aient la même longueur
    for i in range(len(skills_batch)):
        while len(skills_batch[i]) < max_len:
            skills_batch[i].append(0)     # 0 = token "vide"
            weights_batch[i].append(0.0)  # poids nul

    # --- Conversion en tenseurs PyTorch
    return (
        torch.tensor(skills_batch), 
        torch.tensor(weights_batch, dtype=torch.float), 
        torch.tensor(pos_jobs_batch), 
        torch.tensor(neg_jobs_batch)
    ) 

print("Fonction de génération de batch définie")


In [ ]:
# ---------------------------
# 4. Loss contrastive
# ---------------------------
def contrastive_ranking_loss(v_p, v_j_pos, v_j_neg, temperature=0.1):
    # v_p      : vecteurs profils (batch_size, emb_dim)
    # v_j_pos  : vecteurs métiers positifs (batch_size, emb_dim)
    # v_j_neg  : vecteurs métiers négatifs (batch_size, n_neg, emb_dim)
    # temperature : facteur d’échelle pour adoucir ou accentuer les différences

    # Similarité profil / métier positif 
    # Produit scalaire entre le profil et son vrai métier
    pos_sim = torch.sum(v_p * v_j_pos, dim=1, keepdim=True)  
    # shape : (batch_size, 1)

    # Similarité profil / métiers négatifs
    # Produit matriciel entre chaque métier négatif et le profil
    neg_sim = torch.bmm(v_j_neg, v_p.unsqueeze(-1)).squeeze(-1)  
    # shape : (batch_size, n_neg)

    # Concaténation des scores (positif + négatifs)
    logits = torch.cat([pos_sim, neg_sim], dim=1) / temperature  
    # shape : (batch_size, 1 + n_neg)

    # Labels : 0 correspond au vrai métier (position du positif)
    labels = torch.zeros(v_p.size(0), dtype=torch.long, device=v_p.device)

    # Perte de classification (cross-entropy)
    # Le modèle apprend à classer "0" (le vrai métier) comme le plus probable
    return F.cross_entropy(logits, labels)  

print("Fonction de perte définie")


In [ ]:
# ---------------------------
# 5. Entraînement et sauvegarde sur le bucket Supabase (Attention : cela écrase le modèle existant si le nom est identique)
# ---------------------------

# Choix du device (GPU si dispo sinon CPU) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialisation du modèle et de l’optimiseur ---
model = JobProfileTransformer(n_skills, n_jobs, emb_dim=64).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

# Nombre d’époques d’entraînement 
n_epochs = 10   # (test rapide, ton vrai entraînement était sur 4000 epochs)

for epoch in range(n_epochs):
    # --- Génération d’un batch aléatoire 
    skills_batch, weights_batch, pos_jobs, neg_jobs = make_batch(
        batch_size=128,   # nombre d’échantillons par batch
        n_neg=20,         # métiers négatifs par profil
        n_profiles_per_job=5
    )

    # Envoi des données sur GPU/CPU
    skills_batch, weights_batch = skills_batch.to(device), weights_batch.to(device)
    pos_jobs, neg_jobs = pos_jobs.to(device), neg_jobs.to(device)

    # Encodage profil et métiers
    v_p = model.encode_profile(skills_batch, weights_batch)     # (batch_size, emb_dim)
    v_j_pos = model.encode_job(pos_jobs)                        # vrai métier
    v_j_neg = model.encode_job(neg_jobs).view(v_p.size(0), 20, -1)  # métiers négatifs

    # Calcul de la loss contrastive 
    loss = contrastive_ranking_loss(v_p, v_j_pos, v_j_neg)

    # Backpropagation 
    opt.zero_grad()
    loss.backward()
    opt.step()

    # Affichage (ici tous les 100 epochs, mais tu es sur 10 seulement)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, loss={loss.item():.4f}")

# ---------------------------
# Sauvegarde du modèle
# ---------------------------

# Sauvegarde du modèle dans un buffer en mémoire
buffer = io.BytesIO()
torch.save(model.state_dict(), buffer)   # on sauvegarde les poids (state_dict)
buffer.seek(0)
model_bytes = buffer.getvalue()          # conversion en bytes pour upload

# Nom du fichier sauvegardé dans Supabase
name_file = 'job_transformer_test.pth'  #  changer si tu veux éviter l’écrasement

# Vérification si un fichier du même nom existe déjà
existing_files = supabase.storage.from_('DL').list()
if any(f['name'] == name_file for f in existing_files):
    # Suppression de l’ancien fichier
    supabase.storage.from_('dlhybride').remove([name_file])

# Upload du nouveau modèle dans Supabase (upsert = écrase si existe déjà)
supabase.storage.from_('dlhybride').upload(
    name_file, 
    model_bytes,
    {'content-type': 'application/octet-stream', 'upsert': 'true'}
)

print("Modèle sauvegardé dans le bucket DL de Supabase")



# ---------------------------
# Résultats d’entraînement
# ---------------------------
#Epoch 1000, loss=0.2929 22min 52sec
#Epoch 1500, loss=0.1680 28min 27sec
#Epoch 2000, loss=0.1226 34min 10sec
#Epoch 2500, loss=0.0817 50min 16sec
#Epoch 3000, loss=0.0707 59min 01sec
#Epoch 3500, loss=0.0492 59min 41sec
#Epoch 4000, loss=0.0465 69min 11sec



In [ ]:

# ---------------------------
# 6. Rechargement du modèle
# ---------------------------

# Paramètres du bucket et du fichier à télécharger
bucket_name = SUPABASE_BUCKET
file_name = "job_transformer_4000.pth"  #  changer le nom selon le modèle que tu veux recharger

# Téléchargement du modèle depuis Supabase
res = supabase.storage.from_(bucket_name).download(file_name)

# Vérification : si le fichier est vide ou absent → erreur
if res is None or len(res) == 0:
    raise Exception("Erreur téléchargement : fichier non trouvé ou vide")

# Écriture temporaire du fichier en local
import tempfile
with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
    tmp_file.write(res)                 # on écrit les bytes téléchargés
    tmp_model_path = tmp_file.name      # chemin du fichier temporaire

# Création d’un modèle identique à l’original
# il faut utiliser la même définition (n_skills, n_jobs, emb_dim)
model_loaded = JobProfileTransformer(n_skills, n_jobs, emb_dim=64).to(device)

# Chargement des poids sauvegardés
model_loaded.load_state_dict(torch.load(tmp_model_path, map_location=device))

# Passage en mode évaluation (désactive dropout, batchnorm, etc.)
model_loaded.eval()

print("Modèle rechargé et prêt pour simulation ")


In [ ]:
# ---------------------------
# 7. Création de 25 faux profils + upload Supabase
# ---------------------------

# Construction des mappings métiers → compétences et labels
job_to_skills = df.groupby("code_rome")["code_ogr_competence"].apply(list).to_dict()
job_labels = df.drop_duplicates("code_rome").set_index("code_rome")["libelle_rome"].to_dict()
skill_labels = df.drop_duplicates("code_ogr_competence").set_index("code_ogr_competence")["libelle_competence"].to_dict()

# Fonction pour générer un faux profil à partir d'un métier
def generate_fake_profile(job_code, min_ratio=0.5, max_ratio=0.9): 
    """
    Prend un métier (job_code) et retourne une sous-liste de compétences.
    min_ratio/max_ratio : proportion de compétences à garder
    """
    skills = job_to_skills[job_code]                  # compétences du métier
    keep_ratio = random.uniform(min_ratio, max_ratio) # ratio aléatoire
    n_keep = max(1, int(len(skills) * keep_ratio))    # nombre de compétences gardées
    selected = random.sample(skills, n_keep)          # tirage aléatoire
    return selected

# Générer 25 faux profils
fake_profiles = []
for _ in range(25):
    job = random.choice(list(job_to_skills.keys()))  # métier aléatoire
    selected_skills = generate_fake_profile(job)
    fake_profiles.append({
        "job_code": job,
        "job_label": job_labels.get(job, "?"),
        "skills": selected_skills,
        "skills_labels": [skill_labels.get(s, "?") for s in selected_skills]
    })

# Transformation en format tableau pour CSV
rows = []
for i, profile in enumerate(fake_profiles, 1):
    for skill, label in zip(profile["skills"], profile["skills_labels"]):
        rows.append({
            "profile_id": i,
            "job_code": profile["job_code"],
            "job_label": profile["job_label"],
            "skill_code": skill,
            "skill_label": label
        })

df_profiles = pd.DataFrame(rows)

# ---------------------------
# Upload vers Supabase (écrasement autorisé)
# ---------------------------
def upload_df_to_supabase(df, file_name="fake_profiles.csv", bucket_name=SUPABASE_BUCKET):
    try:
        # Sauvegarde en mémoire
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False, encoding="utf-8")
        file_bytes = csv_buffer.getvalue().encode("utf-8")

        # Upload dans le bucket Supabase
        supabase.storage.from_(bucket_name).upload(
            path=file_name,
            file=file_bytes,
            file_options={"content-type": "text/csv", "upsert": "true"}  # upsert = écrase si existant
        )

        print(f"Fichier '{file_name}' uploadé (remplacé si déjà existant) dans le bucket '{bucket_name}'")
    except Exception as ex:
        print("Erreur lors de l'upload :")
        print("->", ex)

# Exécution de l’upload
upload_df_to_supabase(df_profiles, "fake_profiles.csv")



In [ ]:

# ---------------------------
# 8. Charger un CSV depuis Supabase
# ---------------------------
def load_csv_from_supabase(file_name, bucket_name="dlhybride"):
    try:
        files = supabase.storage.from_(bucket_name).list()
    
        response = supabase.storage.from_(bucket_name).download(file_name)
        if response is None or len(response) == 0:
            print(f"Le fichier '{file_name}' est vide ou inexistant.")
            return None
        
        df = pd.read_csv(io.BytesIO(response), dtype=str)
        return df
    
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {file_name} depuis Supabase : {e}")
        return None

# ---------------------------
# Charger df_competence_rome_eda_v2.csv
# ---------------------------
df_jobs = load_csv_from_supabase("df_competence_rome_eda_v2.csv")
if df_jobs is None:
    raise FileNotFoundError("Impossible de charger df_competence_rome_eda_v2.csv")

df_jobs['code_ogr_competence'] = df_jobs['code_ogr_competence'].astype(str)

# Mappings compétences
skills_vocab = {code: idx for idx, code in enumerate(df_jobs['code_ogr_competence'].unique())}
skill_to_label = df_jobs.drop_duplicates('code_ogr_competence') \
                        .set_index('code_ogr_competence')['libelle_competence'].to_dict()

# Mappings métiers
jobs_vocab = {rome: idx for idx, rome in enumerate(df_jobs['code_rome'].unique())}
job_labels = df_jobs.drop_duplicates('code_rome').set_index('code_rome')['libelle_rome'].to_dict()
job_to_skills = df_jobs.groupby('code_rome')['code_ogr_competence'].apply(set).to_dict()

# ---------------------------
# Fonction predict_hybrid 
# ---------------------------
def predict_hybrid(model, input_skills, skills_vocab, job_to_skills, jobs_vocab, job_labels,
                   top_k=3, seuil=0.3, min_overlap=2):
    device = next(model.parameters()).device
    ids = [skills_vocab[s] for s in input_skills if s in skills_vocab]
    
    if len(ids) == 0:
        return "Indéfini (aucune compétence reconnue)"

    skills = torch.tensor(ids).unsqueeze(0).to(device)
    weights = torch.tensor([1.0 for _ in ids], dtype=torch.float).unsqueeze(0).to(device)
    
    v_p = model.encode_profile(skills, weights)
    all_jobs = torch.arange(len(jobs_vocab)).to(device)
    v_j = model.encode_job(all_jobs)
    
    scores_dl = (v_p @ v_j.T).squeeze(0)
    
    input_set = set(input_skills)
    overlap_scores_list = [len(input_set & set(job_to_skills.get(j, []))) for j in jobs_vocab.keys()]
    
    if len(overlap_scores_list) == 0:
        combined_scores = scores_dl
    else:
        overlap_scores = torch.tensor(overlap_scores_list, device=device)
        combined_scores = 0.3 * scores_dl + 0.7 * (overlap_scores / max(1, max(overlap_scores)))

    mask = overlap_scores_list and overlap_scores >= min_overlap if len(overlap_scores_list) > 0 else torch.ones_like(scores_dl, dtype=torch.bool)
    filtered_indices = torch.arange(len(jobs_vocab), device=device)[mask]
    filtered_scores = combined_scores[mask]
    
    if len(filtered_scores) == 0:
        return "Indéfini (aucune compétence ne passe le filtre)"

    best_scores, best_idx = filtered_scores.topk(min(top_k, len(filtered_scores)))
    best_jobs = [list(jobs_vocab.keys())[i] for i in filtered_indices[best_idx]]

    lines = []
    for rome, s in zip(best_jobs, best_scores):
        libelle = job_labels.get(rome, "?")
        # detach() pour éviter le warning PyTorch
        lines.append(f"{rome} - {libelle} - {round(float(s.detach().cpu())*100,1)}%")
    
    if best_scores[0] < seuil:
        return "Indéfini\n" + "\n".join(lines)
    return "\n".join(lines)


In [ ]:

# ---------------------------
# 9. Charger et nettoyer les profils
# ---------------------------
def load_fake_profiles_from_supabase(file_name="fake_profiles.csv", bucket_name=SUPABASE_BUCKET):
    if bucket_name is None or len(bucket_name) < 3:
        bucket_name = "dlhybride"
        print(f"Bucket name trop court ou absent, utilisation de '{bucket_name}'")
    try:
        files = supabase.storage.from_(bucket_name).list()
        
        response = supabase.storage.from_(bucket_name).download(file_name)
        if response is None or len(response) == 0:
            print(f"Le fichier '{file_name}' n'existe pas ou est vide dans le bucket '{bucket_name}'")
            return None
        df = pd.read_csv(io.BytesIO(response), dtype=str)
        df['skill_code'] = df['skill_code'].str.strip()  # Supprimer espaces éventuels
        return df
    except Exception as ex:
        print("Erreur lors de la lecture depuis Supabase :", ex)
        return None

fake_profiles = load_fake_profiles_from_supabase()

# ---------------------------
# Vérifier compétences reconnues
# ---------------------------
skills_vocab_clean = {k.strip(): v for k,v in skills_vocab.items()}  # enlever espaces
df_jobs['code_ogr_competence'] = df_jobs['code_ogr_competence'].str.strip()

# Mapping code -> label
skill_to_label = df_jobs.drop_duplicates('code_ogr_competence') \
                        .set_index('code_ogr_competence')['libelle_competence'].to_dict()

# ---------------------------
# Boucle de prédiction par profil avec comparaison au métier attendu
# ---------------------------
for profile_id in fake_profiles['profile_id'].unique():
    subset = fake_profiles[fake_profiles['profile_id'] == profile_id]
    user_skills = subset['skill_code'].tolist()

    # Compétence attendue (colonne à adapter si elle s'appelle différemment dans ton CSV)
    expected_job = subset['job_code'].iloc[0] if 'job_code' in subset.columns else None

    # Identifier compétences reconnues et non reconnues
    recognized_skills = [s for s in user_skills if s in skills_vocab_clean]
    unrecognized_skills = [s for s in user_skills if s not in skills_vocab_clean]

    user_skills_named = [f"{c} - {skill_to_label.get(c,'?')}" for c in recognized_skills]

    print("="*50)
    print(f"Profil {profile_id} → compétences ({len(user_skills)}):")
    for s in user_skills_named:
        print(f"   • {s}")
    if unrecognized_skills:
        print(f"Compétences non reconnues ({len(unrecognized_skills)}): {unrecognized_skills}")

    if expected_job:
        print(f"\nMétier attendu : {expected_job} - {job_labels.get(expected_job,'?')}")

    print("\nTop-3 métiers proposés :")
    try:
        prediction = predict_hybrid(
            model, recognized_skills,
            skills_vocab_clean, job_to_skills, jobs_vocab, job_labels,
            top_k=3
        )
        print(prediction)
    except Exception as e:
        print(f"Erreur lors de la prédiction : {e}")

    print("\nPrédictions terminées\n")